# Emulation
Here we show how to use one of our trained emulators to generate a mock covariance matrix, and in the end we also demonstrate the power of just-in-time (JIT) compilation in JAX.

In [1]:
import numpy # just to load some files and make the JAX-compatible
import jax
import jax.numpy as np # we assume you want to use jax, just remove "jax." in case you don't
from velocemu.emulator import IntegralEmu
from velocemu.utils import reconstruct_symmetric_matrix_from_lower_diagonal

/home/users/p/piras/veloce/velocemu/emulator.py:6: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
# define the usual cosmological parameters
Omat, H0, lnAs = 0.35, 70., 3.1

In [3]:
# need to provide a file with all redshift pairs and angles;
# since this is a simple tutorial, we produce a dataset on-the-fly made only of 4 objects

z1  = 0.1
z2  = 0.2
z3  = 1.2
z4  = 1.3
a12 = 1.0 # rad
a13 = 1.1 # rad
a14 = 0.7 # rad
a23 = 0.1 # rad
a24 = 2.0 # rad
a34 = 0.1 # rad

z_z_alpha = np.array([[z1, z1, 0.],
                            [z2, z1, a12],
                            [z2, z2, 0.],
                            [z3, z1, a13],
                            [z3, z2, a23],
                            [z3, z3, 0.],
                            [z4, z1, a14],
                            [z4, z2, a24],
                            [z4, z3, a34],
                            [z4, z4, 0.]
                          ])

In [4]:
z_z_alpha

Array([[0.1, 0.1, 0. ],
       [0.2, 0.1, 1. ],
       [0.2, 0.2, 0. ],
       [1.2, 0.1, 1.1],
       [1.2, 0.2, 0.1],
       [1.2, 1.2, 0. ],
       [1.3, 0.1, 0.7],
       [1.3, 0.2, 2. ],
       [1.3, 1.2, 0.1],
       [1.3, 1.3, 0. ]], dtype=float32)

In [5]:
# you should change the relative path to point to your clone of this repository
integrator = IntegralEmu(z_z_alpha) 

# this is just to make everything compilable by JAX
# i.e. a quick fix
unique_rows = numpy.load('./test_unique_rows.npy')
inverse_indices = numpy.load('./test_inverse_indices.npy')
condition = numpy.load('./test_condition.npy')

2025-07-09 01:36:34.737805: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [6]:
cova = reconstruct_symmetric_matrix_from_lower_diagonal(integrator.predict(np.array([Omat,H0,lnAs]),
                                                                           unique_rows, 
                                                                           inverse_indices, 
                                                                           condition,
                                                                           dim1=np.sum(condition),
                                                                           dim2=np.sum(~condition)), 
                                                        4) # 4 is the number of objects

In [7]:
# and in this way we produced the covariance matrix for these 4 objects
print(cova)

[[ 3.5378398e+04  1.5122098e+02 -1.9066612e+00 -2.0086887e+00]
 [ 1.5122098e+02  3.5085820e+04 -6.5791621e+00  1.6719382e+00]
 [-1.9066612e+00 -6.5791621e+00  2.5528650e+04 -8.8573708e+01]
 [-2.0086887e+00  1.6719382e+00 -8.8573708e+01  2.4633096e+04]]


In this repository, we provide the models that lead to the final results, namely the nonlinear case with fixed $\sigma_{\rm u}$. If you are interested in other models, please [raise an issue](https://github.com/dpiras/veloce/issues) or contact [Davide Piras](mailto:dr.davide.piras@gmail.com). Also note that it should be straightforward for you to train your own models using [CosmoPower](https://github.com/alessiospuriomancini/cosmopower), and then add them under `velocemu/trained_models`.

Also note that so far we ignored the possibility to JIT-compile the function to massively speed up the computation of the covariance matrix. To do so, we need to compile the function. We show this below.

In [8]:
import time
start = time.time()
@jax.jit
def predict_jit(Omat,H0,lnAs):
    return reconstruct_symmetric_matrix_from_lower_diagonal(integrator.predict(np.array([Omat,H0,lnAs]),
                                                   unique_rows, 
                                                   inverse_indices, 
                                                   condition,
                                                   # to compile we hardcode these,
                                                   # there is probably a better way but for now
                                                   # this will do
                                                   dim1=4,#np.sum(condition),
                                                   dim2=6),#np.sum(~condition)), 
                                                        4) # 3 is the number of objects

cova=predict_jit(Omat,H0,lnAs)
print(time.time()-start)

0.39968109130859375


In [10]:
import time
start = time.time()

cova=predict_jit(Omat,H0,lnAs)
print(time.time()-start)
print(cova) # obtaining the same results

0.0001347064971923828
[[ 3.5378336e+04  1.5122116e+02 -1.9063989e+00 -2.0086613e+00]
 [ 1.5122116e+02  3.5085770e+04 -6.5793257e+00  1.6719749e+00]
 [-1.9063989e+00 -6.5793257e+00  2.5528605e+04 -8.8573303e+01]
 [-2.0086613e+00  1.6719749e+00 -8.8573303e+01  2.4633037e+04]]


In this case, it took ~100 ms vs ~0.5 s the first time; this corresponds to a speed-up of almost 5000x for every new generation of the covariance matrix!